In [5]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim 
import torch.nn as nn
from torch.utils.data import (
    DataLoader,
)
from tqdm import tqdm

In [6]:
VGG16 = [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']

In [14]:
class VGG_net(nn.Module):
    
    def __init__(self,in_channels=3,num_classes=1000):
        super(VGG_net,self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG16)
        
        self.fully_connected_layers = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Linear(4096,num_classes)
        )
        
    def forward(self,x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fully_connected_layers(x)
        return x
    
    def create_conv_layers(self,architecture):
        layers = []
        
        for x in architecture:
            if type(x) == int:
                out_channels = x
                layers += [
                    nn.Conv2d(self.in_channels,out_channels,kernel_size=3,stride=1,padding=1),
                    nn.ReLU()
                ]
                self.in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
        
        return nn.Sequential(*layers)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG_net(in_channels=3,num_classes=1000).to(device)
X = torch.randn(1,3,224,224).to(device)
print(model(X).shape)

torch.Size([1, 1000])
